# VacationPy

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt 

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yellowknife,62.4560,-114.3525,15.31,82,0,6.17,CA,1691504760
1,1,luderitz,-26.6481,15.1594,22.76,45,0,10.26,NaN,1691504760
2,2,port-aux-francais,-49.3500,70.2167,1.71,78,92,16.73,TF,1691504760
3,3,warburg,51.5000,9.1667,18.17,88,75,4.63,DE,1691504760
4,4,ntoum,0.3833,9.7833,26.87,52,66,4.13,GA,1691504760


### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    hover_cols = ["Max Temp", "Wind Speed", "Cloudiness"],
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.75,
    color = "City"
)

# Display the map
city_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Max Temp,Wind Speed,Cloudiness)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Conditions:
#    A max temperature lower than 26 degrees but higher than 20
#    Wind speed less than 4 m/s
#    Humidity less than 80%
#    Cloudiness less than 30%
target_cities_df = city_data_df.loc[(city_data_df["Max Temp"] > 20) & (city_data_df["Max Temp"] < 26)\
                                    & (city_data_df["Wind Speed"] < 4)\
                                    & (city_data_df["Humidity"] < 80)\
                                    & (city_data_df["Cloudiness"] < 30),:].reset_index(drop=True)


# Drop any rows with null values
target_cities_df = target_cities_df.dropna(how="any")

# Display sample data
target_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,16,hamilton,39.1834,-84.5333,24.32,73,20,2.06,US,1691504525
1,108,racine,42.7495,-88.0751,24.56,76,0,0.89,US,1691504648
2,138,klyuchi,52.2667,79.1667,22.72,43,27,2.32,RU,1691504779
3,232,lensk,60.7253,114.9278,20.68,72,25,1.90,RU,1691504789
4,309,voinesti,47.0667,27.4167,24.82,43,0,3.09,RO,1691504798


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = target_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,hamilton,US,39.1834,-84.5333,73,
1,racine,US,42.7495,-88.0751,76,
2,klyuchi,RU,52.2667,79.1667,43,
3,lensk,RU,60.7253,114.9278,72,
4,voinesti,RO,47.0667,27.4167,43,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "limit":30,
    "apiKey":geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address['features'][0]['properties']['name']
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
hamilton - nearest hotel: North Vista Manor
racine - nearest hotel: No hotel found
klyuchi - nearest hotel: No hotel found
lensk - nearest hotel: No hotel found
voinesti - nearest hotel: No hotel found
belgrade - nearest hotel: Хилтон Београд
murmansk - nearest hotel: No hotel found
evanston - nearest hotel: Margarita European Inn
saint-francois - nearest hotel: Chez Lily
karratha - nearest hotel: Karratha International Hotel
newman - nearest hotel: No hotel found
marquette - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,hamilton,US,39.1834,-84.5333,73,North Vista Manor
1,racine,US,42.7495,-88.0751,76,No hotel found
2,klyuchi,RU,52.2667,79.1667,43,No hotel found
3,lensk,RU,60.7253,114.9278,72,No hotel found
4,voinesti,RO,47.0667,27.4167,43,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# Note: I am filtering out cities where no hotels were found
hotel_plot = hotel_df.loc[hotel_df["Hotel Name"] != "No hotel found"].hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1.5,
    color = "City",
    hover_cols = ["Country", "Hotel Name"]
)

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)